In [ ]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil
import random
from PIL import Image
from ipywidgets import interact, IntSlider
from ultralytics import YOLO

# Creating Contours

In [ ]:
input_dir = 'Contours/original images/images'
output_dir = 'Contours/original images/labels'

In [ ]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def draw_contour(event, x, y, flags, param):
    global drawing, current_contour

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        current_contour.append((x, y))
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            current_contour.append((x, y))
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        current_contour.append((x, y))

def convert_to_yolo_format(contour, img_width, img_height):
    yolo_contour = []
    for point in contour:
        x = point[0] / img_width
        y = point[1] / img_height
        yolo_contour.append(x)
        yolo_contour.append(y)
    return yolo_contour

# Function to load the list of already processed images
def load_processed_images(output_dir):
    processed_images = set()
    for label_file in os.listdir(output_dir):
        if label_file.endswith('.txt'):
            processed_images.add(label_file.replace('.txt', ''))
    return processed_images

# Load the list of already processed images
processed_images = load_processed_images(output_dir)

# Iterate through images in the input directory
for image_name in os.listdir(input_dir):
    image_base_name = os.path.splitext(image_name)[0]
    
    if image_base_name in processed_images:
        print(f"Skipping {image_name}, already processed.")
        continue

    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_dir, image_name)
        image = cv2.imread(image_path)
        img_height, img_width = image.shape[:2]
        
        contours = []
        current_contour = []
        drawing = False

        # Create a window and set a mouse callback to draw the contour
        cv2.namedWindow(image_name)
        cv2.setMouseCallback(image_name, draw_contour)

        while True:
            temp_image = image.copy()
            for contour in contours:
                cv2.polylines(temp_image, [np.array(contour)], isClosed=True, color=(0, 255, 0), thickness=2)
            
            if len(current_contour) > 1:
                cv2.polylines(temp_image, [np.array(current_contour)], isClosed=True, color=(255, 0, 0), thickness=2)
            
            cv2.imshow(image_name, temp_image)
            key = cv2.waitKey(1) & 0xFF

            if key == ord('n'):  # Press 'n' to finalize the current contour and start a new one
                if len(current_contour) > 2:  # Ensure there is a valid contour
                    contours.append(current_contour)
                current_contour = []

            elif key == ord('q'):  # Press 'q' to save and move to the next image
                if len(current_contour) > 2:  # Finalize the last contour if it's valid
                    contours.append(current_contour)
                
                if contours:  # Ensure there are contours to save
                    label_path = os.path.join(output_dir, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))
                    with open(label_path, 'w') as f:
                        for contour in contours:
                            yolo_contour = convert_to_yolo_format(contour, img_width, img_height)
                            f.write(f'0 {" ".join(map(str, yolo_contour))}\n')
                break

            elif key == ord('r'):  # Press 'r' to reset the current contour
                current_contour = []

        cv2.destroyAllWindows()

print("Contour drawing complete.")

## Adding images to labelled images

In [ ]:
base_dir = 'Contours/original images/'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')
labelled_images_dir = os.path.join(base_dir, 'labelled_images')

os.makedirs(labelled_images_dir, exist_ok=True)

In [ ]:
for image_file in os.listdir(images_dir):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):
        # Generate the corresponding label file name
        label_file = os.path.splitext(image_file)[0] + '.txt'
        
        # Check if the corresponding label file exists in the labels directory
        if os.path.exists(os.path.join(labels_dir, label_file)):
            # Copy the image to the labelled_images directory
            shutil.copy(os.path.join(images_dir, image_file), os.path.join(labelled_images_dir, image_file))

print("Images with corresponding labels have been copied to the labelled_images directory.")

In [ ]:
labels_dir = 'Contours/original images/valid/labels'  # Directory containing the labels
labelled_images_dir = 'Contours/original images/labelled_images'  # Directory containing the images
target_images_dir = 'Contours/original images/valid/images'  # Directory to copy the images to

# Ensure the target directory exists
os.makedirs(target_images_dir, exist_ok=True)

# Get the list of labels (assuming each label file corresponds to an image)
label_files = os.listdir(labels_dir)

# Iterate over each label file
for label_file in label_files:
    # Derive the corresponding image name (assuming the same name with .jpg extension)
    image_name = label_file.replace('.txt', '.jpg')
    
    # Construct full image path
    image_path = os.path.join(labelled_images_dir, image_name)
    
    # Check if the image exists in the labelled_images directory
    if os.path.exists(image_path):
        # Copy the image to the target directory
        shutil.copy(image_path, os.path.join(target_images_dir, image_name))
        print(f"Copied: {image_name}")
    else:
        print(f"Image not found for label: {label_file}")